In [7]:
from distribution import SampledDistribution
import numpy as np
from dataclasses import dataclass
from markov_process import *

In [8]:
@dataclass(frozen=True)
class InventoryState:
    on_hand: int
    on_order: int
    def inventory_position(self) -> int:
        return self.on_hand + self.on_order
class SimpleInventoryMRP(MarkovRewardProcess[InventoryState]):
    def __init__(
        self,
        capacity: int,
        poisson_lambda: float,
        holding_cost: float,
        stockout_cost: float
        ):
        self.capacity = capacity
        self.poisson_lambda: float = poisson_lambda
        self.holding_cost: float = holding_cost
        self.stockout_cost: float = stockout_cost
    def transition_reward(
        self,
        state: NonTerminal[InventoryState]
        ) -> SampledDistribution[Tuple[State[InventoryState], float]]:
        def sample_next_state_reward(state=state) ->\
            Tuple[State[InventoryState], float]:
            demand_sample: int = np.random.poisson(self.poisson_lambda)
            ip: int = state.state.inventory_position()
            next_state: InventoryState = InventoryState(
            max(ip - demand_sample, 0),
            max(self.capacity - ip, 0)
            )
            reward: float = - self.holding_cost * state.on_hand\
            - self.stockout_cost * max(demand_sample - ip, 0)
            return NonTerminal(next_state), reward
    
        return SampledDistribution(sample_next_state_reward)

In [11]:
sim = SimpleInventoryMRP(capacity=8,poisson_lambda=1,holding_cost=0.001,stockout_cost= 0.01)

In [13]:
sim.simulate_reward(SampledDistribution())

TypeError: __init__() missing 1 required positional argument: 'sampler'